### Trabalho Prático 2 – Clusterização

#### Importações

In [55]:
!pip install ucimlrepo

In [56]:
import numpy as np
from ucimlrepo import fetch_ucirepo
import time
import warnings
from itertools import cycle, islice
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import adjusted_rand_score

 #### Distância de Minkowski

In [57]:
# p = 1 (Distância Manhattan)
# p = 2 (Distância Euclidiana)

def minkowski(x, y, p):
   return np.power(np.sum(np.abs(x - y) ** p), 1/p)

In [58]:
def matriz_dist(df, p):
    npontos = df.shape[0]
    distancias = np.zeros((npontos, npontos))

    for i in range(npontos):
        for j in range(npontos):
            ponto1 = df.iloc[i].values
            ponto2 = df.iloc[j].values
            distancias[i, j] = minkowski(ponto1, ponto2, p)

    return distancias

#### Algoritmos aproximados K-Centros

##### Algoritmo baseado em refinamento de intervalos

In [59]:
def rvalid(npontos, raio, distancias, k):
  s_aux = np.arange(npontos)
  C = []
  while len(s_aux)>0:
    C.append(s_aux[0])
    s_aux = s_aux[1:]
    s_aux = [x for x in s_aux if distancias[C[-1], x] > 2*raio]
  return len(C) <= k.item(), C

In [60]:
def calculo_rmax(distancias, npontos):
  rmax = -1
  for i in range(npontos):
    for j in range(npontos):
      rmax = max(rmax, distancias[i, j])
  return rmax

In [61]:
def kcentros1(npontos, distancias,k, ref):
  rmax = calculo_rmax(distancias, npontos)
  l = 0;
  r = rmax;

  while((r-l)/rmax > ref):
    m = (l+r)/2.0
    v, C = rvalid(npontos, m, distancias, k)
    if(v):
      r = m
    else:
      l = m+1
  while (l != r):
    m = (l+r)/2
    v, C = rvalid(npontos, m, distancias, k)
    if(v):
      return C
    else:
      l = m+1
  v, C = rvalid(npontos, l, distancias, k)
  return C

Os intervalos considerados serão de 5%, 10%, 15%, 20% e 25%

#### Algoritmo baseado na maximização da distância entre os centros previamente escolhidos.

In [62]:
def kcentros2(k, npontos, distancias):
  if(k.item() > npontos):
    return np.arange()
  C = [0]
  idponto = 0
  while(len(C) < k.item()):
    dist = -1
    for c in C:
      for i in range(npontos):
        if(distancias[i,c] > dist):
          idponto = i
          dist = distancias[i, c]
    C.append(idponto)
  return C

#### Define o indice do cluster para cada um dos dados

In [63]:
def cluster_dados(pontos, centros, distancias):
  labels = []
  for i in range(pontos.shape[0]):
    dist_centro = float('inf')
    id_cluster = 0
    for c in centros:
      if(distancias[i, c] < dist_centro):
        dist_centro = distancias[i, c]
        id_cluster = c
    labels.append(id_cluster)

  return labels

#### Cálculo do raio da solução

In [64]:
def raio(labels, distancias):
  raio = -1
  for i in range(len(labels)):
    raio = max(raio, distancias[i,labels[i]])

  return raio

#### Experimentos com o kcentros1

In [65]:
def experimentos_kcentros1(m_dist, npontos,  X, y_map, k, t_dist, id_dataset):
  ref = [0.05, 0.1, 0.15, 0.2, 0.25]
  results = []
  for i in range(5):
    tmp_exec_kmeans = []
    silhueta_kmeans = []
    indice_rand_kmeans = []
    raio_exec_kmeans = []

    for j in range(30):
      start_time = time.time()

      C = kcentros1(npontos, m_dist, k, ref[i])

      end_time = time.time()
      execution_time = end_time - start_time

      c_dados = cluster_dados(X, C, m_dist)

      tmp_exec_kmeans.append(execution_time)
      if len(np.unique(c_dados)) > 1:
        silhueta_kmeans.append(silhouette_score(X, c_dados))
      else:
        silhueta_kmeans.append(-1)
      indice_rand_kmeans.append(adjusted_rand_score(y_map, c_dados))

      raio_exec_kmeans.append(raio(c_dados, m_dist))

    results.append({
        'Dataset': id_dataset,
        'Algoritmo': 'K-Centros 1',
        'Referência': ref[i],
        'Tempo Execução (s)': f"{np.mean(tmp_exec_kmeans):.4f} ± {np.std(tmp_exec_kmeans):.4f}",
        'Silhueta': f"{np.mean(silhueta_kmeans):.4f} ± {np.std(silhueta_kmeans):.4f}",
        'Índice de Rand Ajustado': f"{np.mean(indice_rand_kmeans):.4f} ± {np.std(indice_rand_kmeans):.4f}",
        'Raio Máximo': f"{np.mean(raio_exec_kmeans):.4f} ± {np.std(raio_exec_kmeans):.4f}"
    })

  df = pd.DataFrame(results)
  df.to_csv(f'dataset_exec_kcentros1_{t_dist}_dataset_{id_dataset}.csv', index=False)

#### Experimentos com o kcentros2

In [66]:
def experimentos_kcentros2(m_dist, npontos, X, y_map, k, t_dist, id_dataset):
    tmp_exec_kmeans = []
    silhueta_kmeans = []
    indice_rand_kmeans = []
    raio_exec_kmeans = []

    for i in range(30):
      start_time = time.time()

      C = kcentros2(k, npontos, m_dist)

      end_time = time.time()
      execution_time = end_time - start_time

      c_dados = cluster_dados(X, C, m_dist)

      tmp_exec_kmeans.append(execution_time)
      silhueta_kmeans.append(silhouette_score(X, c_dados))
      indice_rand_kmeans.append(adjusted_rand_score(y_map, c_dados))

      raio_exec_kmeans.append(raio(c_dados, m_dist))

    data = {
        'Dataset': id_dataset,
        'Algoritmo': 'K-Centros 2',
        'Tempo Execução (s)': f"{np.mean(tmp_exec_kmeans):.4f} ± {np.std(tmp_exec_kmeans):.4f}",
        'Silhueta': f"{np.mean(silhueta_kmeans):.4f} ± {np.std(silhueta_kmeans):.4f}",
        'Índice de Rand Ajustado': f"{np.mean(indice_rand_kmeans):.4f} ± {np.std(indice_rand_kmeans):.4f}",
        'Raio Máximo': f"{np.mean(raio_exec_kmeans):.4f} ± {np.std(raio_exec_kmeans):.4f}"
    }

    df = pd.DataFrame(data, index=[0])
    df.to_csv(f'dataset_exec_kcentros2_{t_dist}_dataset_{id_dataset}.csv', index=False)

#### Avaliação algoritmos aproximados

##### Dados reais

**Pré processamento dos dataset 1**

In [67]:
# Imagens das variedades de passas Kecimen e Besni foram obtidas com CVS.
# Um total de 900 passas foram usadas, incluindo 450 de ambas as variedades, e 7 características morfológicas foram extraídas.

# fetch dataset
raisin = fetch_ucirepo(id=850)

# data (as pandas dataframes)
X = raisin.data.features
y = raisin.data.targets

In [68]:
X = X.dropna()
y = y.loc[X.index]
k = y.nunique()
n_pontos = X.shape[0]
id_dataset = 'Raising'

In [69]:
y['Class'].unique()

array(['Kecimen', 'Besni'], dtype=object)

In [70]:
mapping = {'Kecimen': 0, 'Besni': 1}
y_map = y['Class'].map(mapping)

In [71]:
# Cálculo da matriz de distâncias
p = 1
X_dist_manhattan = matriz_dist(X, p)

In [72]:
# Cálculo matriz de distâncias
p = 2
X_dist_euclidiana = matriz_dist(X, p)

In [73]:
#Aplicação algoritmo k centros 1 distância de Manhattan
experimentos_kcentros1(X_dist_manhattan, n_pontos,  X, y_map, k, 'Manhattan', id_dataset)

# Aplicação algoritmo k centros 1 distância Euclidiana
experimentos_kcentros1(X_dist_euclidiana , n_pontos,  X, y_map, k, 'Euclidiana', id_dataset)

# Aplicação algoritmo kcentros 2 distância de Manhattan
experimentos_kcentros2(X_dist_manhattan, n_pontos, X, y_map, k, 'Manhattan', id_dataset)

# Aplicação algoritmo kcentros 2 distância Euclidiana
experimentos_kcentros2(X_dist_euclidiana , n_pontos, X, y_map, k, 'Euclidiana', id_dataset)

**Pré processamento dataset 2**

In [74]:
# Dados de imagem descritos por atributos numéricos de alto nível, 7 classes

# fetch dataset
image_segmentation = fetch_ucirepo(id=50)

# data (as pandas dataframes)
X = image_segmentation.data.features
y = image_segmentation.data.targets

In [75]:
X = X.dropna()
y = y.loc[X.index]
k = y.nunique()
n_pontos = X.shape[0]
id_dataset = 'Image Segmentation'

In [76]:
y['class'].unique()

array(['BRICKFACE', 'SKY', 'FOLIAGE', 'CEMENT', 'WINDOW', 'PATH', 'GRASS'],
      dtype=object)

In [77]:
mapping = {'BRICKFACE': 0, 'SKY': 1, 'FOLIAGE':2, 'CEMENT': 3, 'WINDOW': 4, 'PATH': 5, 'GRASS': 6}
y_map = y['class'].map(mapping)

In [78]:
# Cálculo da matriz de distâncias
p = 1
X_dist_manhattan = matriz_dist(X, p)

In [79]:
# Cálculo matriz de distâncias
p = 2
X_dist_euclidiana = matriz_dist(X, p)

In [80]:
# Aplicação algoritmo k centros 1 distância de Manhattan
experimentos_kcentros1(X_dist_manhattan, n_pontos,  X, y_map, k, 'Manhattan', id_dataset)

# Aplicação algoritmo k centros 1 distância Euclidiana
experimentos_kcentros1(X_dist_euclidiana , n_pontos,  X, y_map, k,'Euclidiana', id_dataset)

# Aplicação algoritmo kcentros 2 distância de Manhattan
experimentos_kcentros2(X_dist_manhattan, n_pontos, X, y_map , k,'Manhattan', id_dataset)

#Aplicação algoritmo kcentros 2 distância Euclidiana
experimentos_kcentros2(X_dist_euclidiana , n_pontos, X, y_map, k,'Euclidiana', id_dataset)

**Pré processamento dataset 3**

In [81]:
# Dados foram coletados de diferentes hospitais, clínicas comunitárias e unidades
# de saúde materna de áreas rurais de Bangladesh por meio do sistema de monitoramento de risco baseado em IoT.

# fetch dataset
maternal_health_risk = fetch_ucirepo(id=863)

# data (as pandas dataframes)
X = maternal_health_risk.data.features
y = maternal_health_risk.data.targets

In [82]:
X = X.dropna()
y = y.loc[X.index]
k = y.nunique()
n_pontos = X.shape[0]
id_dataset = 'Maternal Health Risk'

In [83]:
y['RiskLevel'].unique()

array(['high risk', 'low risk', 'mid risk'], dtype=object)

In [84]:
mapping = {'high risk': 0, 'low risk': 1, 'mid risk':2}
y_map = y['RiskLevel'].map(mapping)

In [85]:
# Cálculo da matriz de distâncias
p = 1
X_dist_manhattan = matriz_dist(X, p)
p = 2
X_dist_euclidiana = matriz_dist(X, p)

In [86]:
# Aplicação algoritmo k centros 1 distância de Manhattan
experimentos_kcentros1(X_dist_manhattan, n_pontos,  X, y_map, k, 'Manhattan', id_dataset)

# Aplicação algoritmo k centros 1 distância Euclidiana
experimentos_kcentros1(X_dist_euclidiana , n_pontos,  X, y_map, k,'Euclidiana', id_dataset)

# Aplicação algoritmo kcentros 2 distância de Manhattan
experimentos_kcentros2(X_dist_manhattan, n_pontos, X, y_map, k, 'Manhattan', id_dataset)

#Aplicação algoritmo kcentros 2 distância Euclidiana
experimentos_kcentros2(X_dist_euclidiana , n_pontos, X, y_map, k,'Euclidiana', id_dataset)

**Pré processamento dataset 4**

In [87]:
# Discriminação de massas mamográficas benignas e malignas com base nos atributos BI-RADS e na idade do paciente.

# fetch dataset
mammographic_mass = fetch_ucirepo(id=161)

# data (as pandas dataframes)
X = mammographic_mass.data.features
y = mammographic_mass.data.targets

In [88]:
X = X.dropna()
y = y.loc[X.index]
k = y.nunique()
n_pontos = X.shape[0]
id_dataset = 'Mammographic Mass Data'

In [89]:
# Cálculo da matriz de distâncias
p = 1
X_dist_manhattan = matriz_dist(X, p)
p = 2
X_dist_euclidiana = matriz_dist(X, p)

In [90]:
# Aplicação algoritmo k centros 1 distância de Manhattan
experimentos_kcentros1(X_dist_manhattan, n_pontos,  X, y.squeeze(), k,'Manhattan', id_dataset)

# Aplicação algoritmo k centros 1 distância Euclidiana
experimentos_kcentros1(X_dist_euclidiana , n_pontos,  X, y.squeeze(), k, 'Euclidiana', id_dataset)

# Aplicação algoritmo kcentros 2 distância de Manhattan
experimentos_kcentros2(X_dist_manhattan, n_pontos, X, y.squeeze(), k, 'Manhattan', id_dataset)

#Aplicação algoritmo kcentros 2 distância Euclidiana
experimentos_kcentros2(X_dist_euclidiana , n_pontos, X, y.squeeze(),k ,'Euclidiana', id_dataset)

**Pré processamento dataset 5**

In [91]:
# Usamos programas de pré-processamento disponibilizados pelo NIST para
# extrair bitmaps normalizados de dígitos manuscritos de um formulário pré-impresso.

# fetch dataset
optical_recognition_of_handwritten_digits = fetch_ucirepo(id=80)

# data (as pandas dataframes)
X = optical_recognition_of_handwritten_digits.data.features
y = optical_recognition_of_handwritten_digits.data.targets

In [92]:
X = X.dropna()
y = y.loc[X.index]
k = y.nunique()
n_pontos = X.shape[0]
id_dataset = 'Optional Recognition of Handwritten Digits'

In [93]:
# Cálculo da matriz de distâncias
p = 1
X_dist_manhattan = matriz_dist(X, p)
p = 2
X_dist_euclidiana = matriz_dist(X, p)

In [94]:
# Aplicação algoritmo k centros 1 distância de Manhattan
experimentos_kcentros1(X_dist_manhattan, n_pontos,  X, y.squeeze(), k, 'Manhattan', id_dataset)

# Aplicação algoritmo k centros 1 distância Euclidiana
experimentos_kcentros1(X_dist_euclidiana , n_pontos,  X, y.squeeze(), k, 'Euclidiana', id_dataset)

# Aplicação algoritmo kcentros 2 distância de Manhattan
experimentos_kcentros2(X_dist_manhattan, n_pontos, X, y.squeeze(), k, 'Manhattan', id_dataset)

#Aplicação algoritmo kcentros 2 distância Euclidiana
experimentos_kcentros2(X_dist_euclidiana , n_pontos, X, y.squeeze(), k,'Euclidiana', id_dataset)

**Pré processamento dataset 6**

In [95]:
# Objetos 3D dentro de uma imagem 2D pela aplicação de um conjunto de extratores
# de características de forma às silhuetas 2D dos objetos.

# buscar conjunto de dados
statlog_vehicle_silhouettes = fetch_ucirepo(id=149)

# dados (como dataframes pandas)
X = statlog_vehicle_silhouettes.data.features
y = statlog_vehicle_silhouettes.data.targets

In [96]:
X = X.dropna()
y = y.loc[X.index]
k = y.nunique()
n_pontos = X.shape[0]
id_dataset = 'Statlog Vehicle Silhouettes'

In [97]:
y['class'].unique()

array(['van', 'saab', 'bus', 'opel'], dtype=object)

In [98]:
mapping = {'van': 0, 'saab': 1, 'bus':2, 'opel': 3}
y_map = y['class'].map(mapping)

In [99]:
# Cálculo da matriz de distâncias
p = 1
X_dist_manhattan = matriz_dist(X, p)
p = 2
X_dist_euclidiana = matriz_dist(X, p)

In [100]:
# Aplicação algoritmo k centros 1 distância de Manhattan
experimentos_kcentros1(X_dist_manhattan, n_pontos,  X, y_map, k, 'Manhattan', id_dataset)

# Aplicação algoritmo k centros 1 distância Euclidiana
experimentos_kcentros1(X_dist_euclidiana , n_pontos,  X, y_map, k, 'Euclidiana', id_dataset)

# Aplicação algoritmo kcentros 2 distância de Manhattan
experimentos_kcentros2(X_dist_manhattan, n_pontos, X, y_map, k, 'Manhattan', id_dataset)

#Aplicação algoritmo kcentros 2 distância Euclidiana
experimentos_kcentros2(X_dist_euclidiana , n_pontos, X, y_map, k, 'Euclidiana', id_dataset)

**Pré processamento dataset 7**

In [101]:
# Predição dos locais de localização celular de proteínas

# fetch dataset
yeast = fetch_ucirepo(id=110)

# data (as pandas dataframes)
X = yeast.data.features
y = yeast.data.targets

In [102]:
X = X.dropna()
y = y.loc[X.index]
k = y.nunique()
n_pontos = X.shape[0]
id_dataset = 'Yeast'

In [103]:
y['localization_site'].unique()

array(['MIT', 'NUC', 'CYT', 'ME1', 'EXC', 'ME2', 'ME3', 'VAC', 'POX',
       'ERL'], dtype=object)

In [104]:
mapping = {'MIT': 0, 'NUC': 1, 'CYT':2, 'ME1': 3, 'EXC': 4, 'ME2': 5, 'ME3':6, 'VAC': 7, 'POX': 8, 'ERL': 9}
y_map = y['localization_site'].map(mapping)

In [105]:
# Cálculo da matriz de distâncias
p = 1
X_dist_manhattan = matriz_dist(X, p)
p = 2
X_dist_euclidiana = matriz_dist(X, p)

In [106]:
# Aplicação algoritmo k centros 1 distância de Manhattan
experimentos_kcentros1(X_dist_manhattan, n_pontos,  X, y_map, k, 'Manhattan', id_dataset)

# Aplicação algoritmo k centros 1 distância Euclidiana
experimentos_kcentros1(X_dist_euclidiana , n_pontos,  X, y_map, k, 'Euclidiana', id_dataset)

# Aplicação algoritmo kcentros 2 distância de Manhattan
experimentos_kcentros2(X_dist_manhattan, n_pontos, X, y_map, k, 'Manhattan', id_dataset)

#Aplicação algoritmo kcentros 2 distância Euclidiana
experimentos_kcentros2(X_dist_euclidiana , n_pontos, X, y_map, k, 'Euclidiana', id_dataset)

**Pré processamento dataset 8**

In [113]:
# fetch dataset
blood_transfusion_service_center = fetch_ucirepo(id=176)

# data (as pandas dataframes)
X = blood_transfusion_service_center.data.features
y = blood_transfusion_service_center.data.targets


In [114]:
X = X.dropna()
y = y.loc[X.index]
k = y.nunique()
n_pontos = X.shape[0]
id_dataset = 'Blood Transfusion Service Center Data'

In [115]:
# Cálculo da matriz de distâncias
p = 1
X_dist_manhattan = matriz_dist(X, p)
p = 2
X_dist_euclidiana = matriz_dist(X, p)

In [118]:
# Aplicação algoritmo k centros 1 distância de Manhattan
experimentos_kcentros1(X_dist_manhattan, n_pontos,  X, y.squeeze(), k, 'Manhattan', id_dataset)

# Aplicação algoritmo k centros 1 distância Euclidiana
experimentos_kcentros1(X_dist_euclidiana , n_pontos,  X, y.squeeze(), k, 'Euclidiana', id_dataset)

# Aplicação algoritmo kcentros 2 distância de Manhattan
experimentos_kcentros2(X_dist_manhattan, n_pontos, X, y.squeeze(), k, 'Manhattan', id_dataset)

#Aplicação algoritmo kcentros 2 distância Euclidiana
experimentos_kcentros2(X_dist_euclidiana , n_pontos, X, y.squeeze(), k, 'Euclidiana', id_dataset)

**Pré processamento dataset 9**

In [119]:
# Preveja a idade do abalone a partir de medidas físicas

# fetch dataset
abalone = fetch_ucirepo(id=1)

# data (as pandas dataframes)
X = abalone.data.features
y = abalone.data.targets

In [120]:
X = X.drop('Sex', axis = 1)

In [121]:
X = X.dropna()
y = y.loc[X.index]
k = y.nunique()
n_pontos = X.shape[0]
id_dataset = 'Abalone'

In [122]:
# Cálculo da matriz de distâncias
p = 1
X_dist_manhattan = matriz_dist(X, p)
p = 2
X_dist_euclidiana = matriz_dist(X, p)

In [124]:
# Aplicação algoritmo k centros 1 distância de Manhattan
experimentos_kcentros1(X_dist_manhattan, n_pontos,  X, y.squeeze(), k, 'Manhattan', id_dataset)

# Aplicação algoritmo k centros 1 distância Euclidiana
experimentos_kcentros1(X_dist_euclidiana , n_pontos,  X, y.squeeze(), k, 'Euclidiana', id_dataset)

# Aplicação algoritmo kcentros 2 distância de Manhattan
experimentos_kcentros2(X_dist_manhattan, n_pontos, X, y.squeeze(), k, 'Manhattan', id_dataset)

#Aplicação algoritmo kcentros 2 distância Euclidiana
experimentos_kcentros2(X_dist_euclidiana , n_pontos, X, y.squeeze(), k, 'Euclidiana', id_dataset)

**Pré processamento dataset 10**

In [125]:
# Este conjunto de dados contém recursos extraídos do conjunto de
# imagens Messidor para prever se uma imagem contém sinais de retinopatia diabética ou não.

# fetch dataset
diabetic_retinopathy_debrecen = fetch_ucirepo(id=329)

# data (as pandas dataframes)
X = diabetic_retinopathy_debrecen.data.features
y = diabetic_retinopathy_debrecen.data.targets

In [126]:
X = X.dropna()
y = y.loc[X.index]
k = y.nunique()
n_pontos = X.shape[0]
id_dataset = 'Diabetic Retinopathy Debrecen'

In [127]:
# Cálculo da matriz de distâncias
p = 1
X_dist_manhattan = matriz_dist(X, p)
p = 2
X_dist_euclidiana = matriz_dist(X, p)

In [131]:
# Aplicação algoritmo k centros 1 distância de Manhattan
experimentos_kcentros1(X_dist_manhattan, n_pontos,  X, y.squeeze(), k, 'Manhattan', id_dataset)

# Aplicação algoritmo k centros 1 distância Euclidiana
experimentos_kcentros1(X_dist_euclidiana , n_pontos,  X, y.squeeze(), k,  'Euclidiana', id_dataset)

# Aplicação algoritmo kcentros 2 distância de Manhattan
experimentos_kcentros2(X_dist_manhattan, n_pontos, X, y.squeeze(), k, 'Manhattan', id_dataset)

#Aplicação algoritmo kcentros 2 distância Euclidiana
experimentos_kcentros2(X_dist_euclidiana , n_pontos, X, y.squeeze(), k, 'Euclidiana', id_dataset)

In [136]:
import glob
def consolidar_csvs(path, arquivo_saida):
    file_pattern = "dataset_exec_kcentros2_Manhattan_dataset_*.csv"

    files = glob.glob(file_pattern)

    dataframes = []

    for file in files:
        df = pd.read_csv(file)
        dataframes.append(df)

    df_consolidado = pd.concat(dataframes, ignore_index=True)

    df_consolidado.to_csv(arquivo_saida, index=False)

In [137]:
consolidar_csvs('.', 'dataset_kcentros2_Manhattan_dataset_dados_reais.csv')